In [ ]:
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import sys
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.ensemble import RandomForestRegressor # RandomForestRegressor 모델 (Random Forest Regressor model)
from sklearn.model_selection import train_test_split, RandomizedSearchCV # 데이터 분할 및 하이퍼파라미터 튜닝

In [ ]:
# 데이터 로드 모듈 임포트 및 실행 (Import and run data load module)
try:
    import data_load
    print("✅ data_load 모듈 임포트 성공 (data_load module imported successfully)")
    # 데이터 로드 (Load data - start/end dates are examples, modify as needed)
    combined_df = data_load.load_data(start_date='2025-01-01', end_date='2025-09-14')
    if combined_df is None or combined_df.empty:
        print("❌ 데이터 로드 실패 또는 데이터프레임이 비어있습니다. 스크립트를 중단합니다. (Data load failed or DataFrame is empty. Stopping script.)")
        # exit()
except ImportError as e:
    print(f"❌ data_load 모듈 임포트 중 오류 발생 (Error importing data_load module): {e}")
    print(f"'{utils_path}' 경로에 data_load.py 파일이 있는지 확인하세요. (Check if data_load.py exists in '{utils_path}'.)")
    combined_df = None # 오류 시 None으로 초기화 (Initialize to None on error)
    # exit()
except Exception as e:
    print(f"❌ 데이터 로드 중 예상치 못한 오류 발생 (Unexpected error during data load): {e}")
    combined_df = None
    # exit()


✅ data_load 모듈 임포트 성공 (data_load module imported successfully)
🔍 ['2025-01-01'부터 '2025-09-14'까지의 데이터를 로드]


/content/drive/MyDrive/utils/data_load.py:48: DtypeWarning: Columns (19,20,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(
/content/drive/MyDrive/utils/data_load.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs_to_concat, ignore_index=True)



✅ 총 257개 파일, 8474140개 행의 데이터를 성공적으로 합쳤습니다.


In [ ]:
# ## 데이터 전처리 (Data Preprocessing)

# preprocessor.py 모듈을 임포트하고, 로드된 데이터에 전처리 함수를 적용합니다.
# Import the preprocessor.py module and apply the preprocessing function to the loaded data.
preprocessor = None # preprocessor 변수 초기화
expected_preprocessor_path = os.path.join(utils_path, 'preprocessor.py') # 예상 경로 미리 정의
try:
    import preprocessor
    print("✅ preprocessor 모듈 임포트 성공 (preprocessor module imported successfully)")
except ImportError as e:
    print(f"❌ preprocessor 모듈 임포트 중 오류 발생 (Error importing preprocessor module): {e}")
    print(f"'{utils_path}' 경로에 preprocessor.py 파일이 있는지 확인하세요. (Check if preprocessor.py exists in '{utils_path}'.)")
    preprocessor = None # 오류 시 None으로 초기화 (Initialize to None on error)
    # exit()


# 데이터 전처리 실행 (Run data preprocessing)
preprocessed_df = None # 전처리 후 데이터프레임 변수 초기화 (Initialize preprocessed DataFrame variable)
if 'combined_df' in locals() and combined_df is not None and 'preprocessor' in locals() and preprocessor is not None:
    print("\n--- 데이터 전처리 시작 (Starting data preprocessing) ---")

    # Object 타입 컬럼을 문자열로 변환 (Convert object type columns to string)
    # This step is crucial if the preprocessor uses tools like LabelEncoder that expect uniform data types.
    object_cols = combined_df.select_dtypes(include=['object']).columns
    if not object_cols.empty:
        print(f"ℹ️ 다음 object 컬럼을 문자열로 변환합니다 (Converting the following object columns to string): {list(object_cols)}")
        for col in object_cols:
            try:
                # Missing values might cause issues during conversion; consider filling them first if necessary.
                # Example: combined_df[col].fillna('Unknown', inplace=True)
                combined_df[col] = combined_df[col].astype(str)
            except Exception as e:
                 print(f"⚠️ 컬럼 '{col}' 변환 중 오류 (Error converting column '{col}'): {e}. 해당 컬럼 처리를 건너뛸 수 있습니다. (May skip processing this column.)")
        print("✅ Object 타입 컬럼 문자열 변환 완료 (Object type column conversion to string complete - some may be skipped on error)")
    else:
        print("ℹ️ 문자열로 변환할 Object 타입 컬럼이 없습니다. (No object type columns to convert to string.)")

    # 전처리 함수 호출 (Call preprocessing function)
    try:
        # Using copy() is recommended to preserve the original combined_df.
        print("preprocessor.preprocessor 함수 실행 중...")
        preprocessed_df = preprocessor.preprocessor(combined_df.copy())
        print(f"✅ 전처리 함수 실행 완료. 형상: {preprocessed_df.shape} (Preprocessing function execution complete. Shape: {preprocessed_df.shape})")
    except Exception as e:
        print(f"❌ 전처리 함수 실행 중 오류 발생 (Error executing preprocessing function): {e}")
        # exit() # 오류 발생 시 중단 (Exit on error)

    # '효율(순간)' 값이 100 이상인 비정상 데이터 제거 (Remove outliers where '효율(순간)' >= 100)
    if preprocessed_df is not None:
        try:
            target_col = '효율(순간)' # 목표 변수 컬럼명 (Target variable column name)
            if target_col in preprocessed_df.columns:
                initial_rows = len(preprocessed_df)
                # Filter out rows where the target value is abnormally high (>= 100)
                preprocessed_df = preprocessed_df[preprocessed_df[target_col] < 100].copy()
                removed_rows = initial_rows - len(preprocessed_df)
                print(f"✅ '{target_col}' 100 이상인 행 {removed_rows}개 제거 완료 (Removed {removed_rows} rows where '{target_col}' >= 100)")
                print(f"최종 전처리 데이터 형상: {preprocessed_df.shape} (Final preprocessed data shape: {preprocessed_df.shape})")
                print("--- 데이터 전처리 완료 (Data preprocessing complete) ---")
        except Exception as e:
            print(f"❌ '{target_col}' 100 이상 행 제거 중 오류 (Error removing rows where '{target_col}' >= 100): {e}")

✅ preprocessor 모듈 임포트 성공 (preprocessor module imported successfully)

--- 데이터 전처리 시작 (Starting data preprocessing) ---
ℹ️ 다음 object 컬럼을 문자열로 변환합니다 (Converting the following object columns to string): ['생성일', '급수펌프 입력', '배기가스온도2', '배기가스온도3', '배기 재 순환 온도', '에코 온도1', '에코 온도2', '버너온도', '재순환 O2', '재순환 NOx', '운전시간', '확률 업데이트 시간']
✅ Object 타입 컬럼 문자열 변환 완료 (Object type column conversion to string complete - some may be skipped on error)
preprocessor.preprocessor 함수 실행 중...
--- 데이터 전처리 시작 ---
✅ 1. 불필요한 컬럼 14개 제거 완료
✅ 1. 불필요한 컬럼 제거 완료
✅ 2. '효율(순간)' 컬럼 값 100 미만으로 필터링 완료
✅ 3-1. 범주형 데이터 결측치 처리 완료
✅ 3-2. 범주형 변수 변환 완료
ℹ️ 3-3. 처리할 수치형 결측치가 없습니다.
✅ 4. 피처 스케일링 완료
--- 데이터 전처리 완료 ---
✅ 전처리 함수 실행 완료. 형상: (4796529, 33) (Preprocessing function execution complete. Shape: (4796529, 33))
✅ '효율(순간)' 100 이상인 행 0개 제거 완료 (Removed 0 rows where '효율(순간)' >= 100)
최종 전처리 데이터 형상: (4796529, 33) (Final preprocessed data shape: (4796529, 33))
--- 데이터 전처리 완료 (Data preprocessing complete) ---


In [ ]:
# ## 데이터 분할 (Data Splitting)

# 전처리된 데이터를 학습용(Train)과 테스트용(Test)으로 분할합니다 (보통 80:20 비율).
# Split the preprocessed data into training and testing sets (typically 80:20 ratio).
X_train, X_test, y_train, y_test = None, None, None, None # Initialize variables to None
# Proceed only if preprocessing was successful
if 'preprocessed_df' in locals() and preprocessed_df is not None and not preprocessed_df.empty:
    try:
        y_col = '효율(순간)' # Define the name of the dependent variable (target) column
        # Ensure the target column exists before attempting to drop/select it
        if y_col not in preprocessed_df.columns:
            raise KeyError(f"'{y_col}' 컬럼이 전처리된 데이터프레임에 없습니다. (Column '{y_col}' not found in preprocessed DataFrame.)")

        # Separate features (X) and target (y)
        X = preprocessed_df.drop(columns=[y_col]) # Independent variables (Features)
        y = preprocessed_df[y_col]               # Dependent variable (Target)

        # Split data into training and test sets using train_test_split
        # test_size=0.2 means 20% for testing, 80% for training
        # random_state ensures the split is the same every time the code runs (for reproducibility)
        # shuffle=True (default) shuffles data before splitting
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

        # Print shapes of the resulting datasets to confirm the split
        print("\n✅ 데이터 학습/테스트 분할 완료 (Data split into train/test complete) (80%/20%)")
        print(f"학습 데이터 X 형상 (Train X shape): {X_train.shape}, 테스트 데이터 X 형상 (Test X shape): {X_test.shape}")
        print(f"학습 데이터 y 형상 (Train y shape): {y_train.shape}, 테스트 데이터 y 형상 (Test y shape): {y_test.shape}")

    # Handle error if the target column is missing
    except KeyError as e:
        print(f"❌ 데이터 분할 중 오류 발생: {e}")
    # Handle any other unexpected errors during splitting
    except Exception as e:
        print(f"❌ 데이터 분할 중 예상치 못한 오류 발생 (Unexpected error during data split): {e}")


✅ 데이터 학습/테스트 분할 완료 (Data split into train/test complete) (80%/20%)
학습 데이터 X 형상 (Train X shape): (3837223, 32), 테스트 데이터 X 형상 (Test X shape): (959306, 32)
학습 데이터 y 형상 (Train y shape): (3837223,), 테스트 데이터 y 형상 (Test y shape): (959306,)


In [ ]:
# ## 하이퍼파라미터 튜닝 (Hyperparameter Tuning)

# RandomizedSearchCV를 사용하여 Random Forest 모델의 최적 하이퍼파라미터 조합을 탐색합니다.
# Use RandomizedSearchCV to find the optimal combination of hyperparameters for the Random Forest model.
best_params_rf = None # Initialize variable to store best parameters
# Proceed only if training data is available
if X_train is not None and y_train is not None:
    print("\n--- 하이퍼파라미터 튜닝 시작 (Starting Hyperparameter Tuning - RandomizedSearchCV for RandomForest) ---")
    # Initialize the Random Forest Regressor model for tuning
    # random_state for reproducibility, n_jobs=-1 to use all CPU cores
    rf_model_tune = RandomForestRegressor(random_state=42, n_jobs=-1)

    # Define the parameter space (distributions) to sample from during Randomized Search
    param_dist_rf = {
        'n_estimators': np.arange(50, 501, 50),     # Number of trees: 50, 100, ..., 500
        'max_depth': [None] + list(np.arange(5, 31, 5)), # Max depth: None (unlimited), 5, 10, ..., 30
        'min_samples_split': [2, 5, 10, 15],         # Min samples to split a node: 2, 5, 10, 15
        'min_samples_leaf': [1, 3, 5, 7],            # Min samples at a leaf node: 1, 3, 5, 7
        'max_features': ['sqrt', 'log2', 0.5, 0.7] # Max features for split: sqrt(n_features), log2(n_features), 50%, 70%
        # 'bootstrap': [True, False] # Whether bootstrap samples are used (optional)
    }

    # Configure RandomizedSearchCV
    # n_iter: Number of parameter settings that are sampled. Trades off runtime vs quality of the solution.
    # cv: Number of cross-validation folds.
    # scoring: Strategy to evaluate the performance of the cross-validated model on the test set. 'neg_mean_squared_error' is common for regression.
    # verbose: Controls the verbosity: the higher, the more messages.
    # n_jobs=-1: Use all available CPU cores for parallel processing.
    n_iterations = 5 # Number of iterations (parameter combinations to try). Adjust based on available time.
    cv_folds = 2      # Number of cross-validation folds. Adjust based on available time/data size.

    random_search_rf = RandomizedSearchCV(
        estimator=rf_model_tune,           # The model to tune
        param_distributions=param_dist_rf, # The distributions to sample parameters from
        n_iter=n_iterations,               # Number of parameter settings that are sampled
        scoring='neg_mean_squared_error',  # Metric to evaluate the performance (negative MSE)
        cv=cv_folds,                       # Number of cross-validation folds
        verbose=1,                         # Print progress messages
        random_state=42,                   # Seed for reproducibility
        n_jobs=-1                          # Use all available CPU cores
    )

    # Randomized Search 실행 (Execute Randomized Search on the training data)
    try:
        print(f"{n_iterations}개 조합, {cv_folds}-겹 교차 검증으로 Randomized Search 실행... (Running Randomized Search with {n_iterations} combinations and {cv_folds}-fold CV...)")
        # Fit RandomizedSearchCV to find the best parameters
        random_search_rf.fit(X_train, y_train)
        # Store the best parameters found
        best_params_rf = random_search_rf.best_params_
        print("\n✅ 하이퍼파라미터 튜닝 완료 (Hyperparameter tuning complete)")
        print("최적 하이퍼파라미터 (Best hyperparameters - Random Forest):", best_params_rf)
        # The best_score_ attribute gives the score for the best parameters found.
        # Since scoring='neg_mean_squared_error', a higher score (closer to 0) is better.
        print(f"최고 교차 검증 점수 (Best cross-validation score - Negative MSE): {random_search_rf.best_score_:.6f}")
    # Handle potential errors during the tuning process
    except Exception as e:
        print(f"❌ 하이퍼파라미터 튜닝 중 오류 발생 (Error during hyperparameter tuning): {e}")
        best_params_rf = None # Reset best parameters on error



--- 하이퍼파라미터 튜닝 시작 (Starting Hyperparameter Tuning - RandomizedSearchCV for RandomForest) ---
5개 조합, 2-겹 교차 검증으로 Randomized Search 실행... (Running Randomized Search with 5 combinations and 2-fold CV...)
Fitting 2 folds for each of 5 candidates, totalling 10 fits


In [ ]:
 ## 최종 모델 학습 및 평가 (Final Model Training & Evaluation)

# RandomizedSearchCV를 통해 찾은 최적 하이퍼파라미터를 사용하여 최종 Random Forest 모델을 학습시키고, 테스트 데이터로 성능을 평가합니다.
# Train the final Random Forest model using the best hyperparameters found by RandomizedSearchCV and evaluate its performance on the test data.
rf_final_model = None # Initialize final model variable to None
# Proceed only if best parameters were found and training data exists
if best_params_rf is not None and X_train is not None and y_train is not None:
    print("\n--- 최적 파라미터로 Random Forest 모델 학습 및 평가 시작 (Starting final Random Forest model training and evaluation with best parameters) ---")
    # Initialize the final model using the best parameters found
    # The ** operator unpacks the dictionary into keyword arguments
    rf_final_model = RandomForestRegressor(
        **best_params_rf,
        random_state=42, # Ensure reproducibility for the final model
        n_jobs=-1        # Use all CPU cores
    )

    # 모델 학습 (Train the final model on the entire training dataset)
    try:
        print("최적 모델 학습 중... (Training the optimal model...)")
        # Fit the model
        rf_final_model.fit(X_train, y_train)
        print("✅ 모델 학습 완료 (Model training complete)")

        # 예측 수행 (Make predictions on both training and test data)
        print("예측 수행 중... (Making predictions...)")
        y_train_pred_rf = rf_final_model.predict(X_train) # Predictions on training data
        y_test_pred_rf = rf_final_model.predict(X_test)   # Predictions on test data
        print("✅ 예측 수행 완료 (Prediction complete)")

        # 평가 지표 계산 (Calculate various evaluation metrics)
        print("평가 지표 계산 중... (Calculating evaluation metrics...)")
        # R-squared (Coefficient of Determination)
        train_r2_rf = r2_score(y_train, y_train_pred_rf)
        test_r2_rf = r2_score(y_test, y_test_pred_rf)
        # Mean Squared Error (MSE)
        train_mse_rf = mean_squared_error(y_train, y_train_pred_rf)
        test_mse_rf = mean_squared_error(y_test, y_test_pred_rf)
        # Mean Absolute Error (MAE)
        train_mae_rf = mean_absolute_error(y_train, y_train_pred_rf)
        test_mae_rf = mean_absolute_error(y_test, y_test_pred_rf)
        # Root Mean Squared Error (RMSE)
        train_rmse_rf = np.sqrt(train_mse_rf)
        test_rmse_rf = np.sqrt(test_mse_rf)

        # MAPE 계산 함수 정의 (Define MAPE function to handle potential division by zero)
        def mean_absolute_percentage_error(y_true, y_pred):
            # Convert inputs to numpy arrays
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            # Create a mask for non-zero true values to avoid division by zero
            non_zero_mask = y_true != 0
            # If all true values are zero, return NaN
            if np.sum(non_zero_mask) == 0: return np.nan
            # Calculate MAPE only for non-zero true values and take the mean
            return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

        # Calculate Mean Absolute Percentage Error (MAPE)
        train_mape_rf = mean_absolute_percentage_error(y_train, y_train_pred_rf)
        test_mape_rf = mean_absolute_percentage_error(y_test, y_test_pred_rf)

        # 결과 출력 (Print the calculated evaluation metrics)
        print("\n--- Random Forest 모델 평가 결과 (튜닝 후) (Random Forest Model Evaluation Results - Tuned) ---")
        print(f"Train R2: {train_r2_rf:.6f}")
        print(f"Test R2 : {test_r2_rf:.6f}")
        print(f"Train MSE: {train_mse_rf:.6f}")
        print(f"Test MSE : {test_mse_rf:.6f}")
        print(f"Train MAE: {train_mae_rf:.6f}")
        print(f"Test MAE : {test_mae_rf:.6f}")
        print(f"Train RMSE: {train_rmse_rf:.6f}")
        print(f"Test RMSE : {test_rmse_rf:.6f}")
        # Check if MAPE is NaN before formatting
        train_mape_str = f"{train_mape_rf:.4f}%" if not np.isnan(train_mape_rf) else "N/A (y_train contains zeros)"
        test_mape_str = f"{test_mape_rf:.4f}%" if not np.isnan(test_mape_rf) else "N/A (y_test contains zeros)"
        print(f"Train MAPE: {train_mape_str}")
        print(f"Test MAPE : {test_mape_str}")

    # Handle potential errors during final model training or evaluation
    except Exception as e:
        print(f"❌ 모델 학습 또는 평가 중 오류 발생 (Error during model training or evaluation): {e}")
        rf_final_model = None # Reset final model variable on error

In [ ]:
# ## 특성 중요도 분석 (Feature Importance Analysis)

# 학습된 최종 Random Forest 모델의 특성 중요도를 확인하고 시각화합니다.
# Check and visualize the feature importances of the trained final Random Forest model.
# Proceed only if the final model was trained successfully and training data exists
if rf_final_model is not None and X_train is not None:
    print("\n--- Random Forest 특성 중요도 확인 (Checking Random Forest Feature Importances) ---")
    try:
        # Extract feature importances from the trained model
        importances_rf = rf_final_model.feature_importances_
        # Get the names of the features
        feature_names_rf = X_train.columns
        # Create a DataFrame for easier handling and sorting
        feature_importance_df_rf = pd.DataFrame({'feature': feature_names_rf, 'importance': importances_rf})
        # Sort features by importance in descending order
        feature_importance_df_rf = feature_importance_df_rf.sort_values(by='importance', ascending=False)

        # Print the top 10 most important features
        print("상위 10개 특성 중요도 (Top 10 Feature Importances - Random Forest):")
        print(feature_importance_df_rf.head(10))

        # 특성 중요도 시각화 (Visualize feature importances - Top 10)
        plt.figure(figsize=(12, 7)) # Adjust figure size for better readability
        top_n = 10
        # Select top N features and sort them ascending for plotting (makes highest importance appear at the top)
        top_features_rf = feature_importance_df_rf.head(top_n).sort_values(by='importance', ascending=True)

        # Create horizontal bar plot using seaborn for potentially better aesthetics
        sns.barplot(x='importance', y='feature', data=top_features_rf, palette='viridis') # Use a color palette
        # plt.barh(top_features_rf['feature'], top_features_rf['importance'], color='skyblue') # Alternative using Matplotlib

        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.title(f'Top {top_n} Feature Importances (Random Forest - Tuned)')
        plt.tight_layout() # Adjust layout to prevent labels overlapping
        plt.show() # Display the plot


    # Handle potential errors during importance calculation or visualization
    except Exception as e:
        print(f"❌ 특성 중요도 확인 또는 시각화 중 오류 발생 (Error during feature importance check or visualization): {e}")

# Indicate the end of the notebook execution
print("\n--- 모든 작업 완료 (All tasks completed) ---")

In [ ]:
# 모델 평가 (Model Evaluation)
if best_params_rf is not None and X_test is not None and y_test is not None:
    print("\n--- 모델 평가 시작 (Starting Model Evaluation) ---")
    try:
        # 최적 파라미터로 최종 모델 학습 (Train final model with best parameters)
        final_rf_model = RandomForestRegressor(**best_params_rf, random_state=42, n_jobs=-1)
        final_rf_model.fit(X_train, y_train)

        # 테스트 데이터 예측 (Predict on test data)
        y_pred = final_rf_model.predict(X_test)

        # 성능 지표 계산 (Calculate performance metrics)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse) # RMSE
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print("\n✅ 모델 평가 완료 (Model evaluation complete)!")
        print(f"✨ Mean Squared Error (MSE): {mse:.4f}")
        print(f"✨ Root Mean Squared Error (RMSE): {rmse:.4f}")
        print(f"✨ Mean Absolute Error (MAE): {mae:.4f}")
        print(f"✨ R-squared (R2): {r2:.4f}")

    except Exception as e:
        print(f"❌ 모델 평가 중 오류 발생 (Error during model evaluation): {e}")